In [2]:
### Inference on a single image
import os
import json
import torch
import torch.nn as nn
from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s
from PIL import Image

# --- Configuration ---
MODEL_PATH = 'models/best_model.pth'
# IMAGE_PATH = r'dataset\images\test\Hummus\18.jpg'  # <<<--- Change this to your image file
IMAGE_PATH = r"Final_dataset/Zaatar/141.jpg"
MAPPING_JSON_PATH = 'class_mapping.json'
NUM_CLASSES = 66

# --- Device Setup ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- Load Class Mapping ---
print(f"Loading class mapping from: {MAPPING_JSON_PATH}")
try:
    with open(MAPPING_JSON_PATH, 'r') as f:
        class_to_idx = json.load(f)
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    if len(idx_to_class) != NUM_CLASSES:
        print(f"Warning: Found {len(idx_to_class)} classes, expected {NUM_CLASSES}")
except Exception as e:
    raise RuntimeError(f"Failed to load class mapping: {e}")

# --- Load Model ---
print("Loading model...")
weights = EfficientNet_V2_S_Weights.IMAGENET1K_V1
model = efficientnet_v2_s(weights=weights)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Sequential(
    nn.Dropout(p=0.37, inplace=True), # Keep dropout
    nn.Linear(num_ftrs, NUM_CLASSES),
)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# --- Preprocessing ---
preprocess = weights.transforms()

# --- Prediction Function ---
def predict_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        input_tensor = preprocess(img).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            output = model(input_tensor)
            conf, idx = torch.max(torch.softmax(output, dim=1), 1)
        return idx_to_class.get(idx.item(), "Unknown"), conf.item()
    except Exception as e:
        print(f"Prediction error: {e}")
        return None, None

# --- Inference ---
if not os.path.exists(IMAGE_PATH):
    print(f"Image path does not exist: {IMAGE_PATH}")
else:
    print(f"Predicting image: {IMAGE_PATH}")
    label, confidence = predict_image(IMAGE_PATH)
    if label:
        print(f"-> Predicted Class: {label}")
        print(f"-> Confidence: {confidence:.4f}")

Using device: cpu
Loading class mapping from: class_mapping.json
Loading model...
Predicting image: Final_dataset/Zaatar/141.jpg
-> Predicted Class: Zaatar
-> Confidence: 0.9074
